#### Generate EMG recordings using neuromotion through `muniverse`

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from muniverse.data_generation import generate_recording

# Define the configuration file path and output directory
config_path = '../src/data_generation/neuromotion_config_template.json'
output_dir = '../outputs'

run_dir = generate_recording({
    'input_config': config_path,
    'output_dir': output_dir,
    'engine': 'docker',
    'container_name': 'pranavm19/muniverse-test:neuromotion'
})

Optionally, (1) format the data into BIDS structure, and (2)upload them to harvard dataverse

In [3]:
from muniverse.utils import bidsify_data

data_path = run_dir
root = '../'
datasetname = 'neuromotion-dev'
bidsdata = bidsify_data.neuromotion_to_bids(data_path, root, datasetname)

In [ ]:
from muniverse.utils.dataverse import DataverseUploader

DATAVERSE_URL = "https://demo.dataverse.org"
API_TOKEN = "c8c590f1-2f8f-4784-881f-2a1c14417fd1"

# Initialize the uploader
uploader = DataverseUploader(
    api_token=API_TOKEN,
    dataverse_url=DATAVERSE_URL,
    dataverse_alias="muniverse-neuromotion-demo",
)

In [12]:
datasets = uploader.list_datasets()
uploader.set_dataset(datasets[1]['persistentUrl'])

In [ ]:
uploader.upload_directory(
    local_directory="../neuromotion-dev",
    remote_base_directory="neuromotion-dev",
    dry_run=True
)